## Speech Recognition & Word Predictor

### Abstract
This jupyter-notebook is a continuation of the Google Colab(Song Builder & Lyric Predictor) built prior to this using LSTM,
Embedding and Dense layers. Refer https://colab.research.google.com/drive/1Hgxl6JWOuTnbuIXYrTLabTzvx2CSstLG#scrollTo=IYB2zxaczkQf.
In this, we will be predicting the next words, specifically song lyrics to user audio input and returning it in an audio format instead of text. As our model is only 83% accurate and trained on a restricted dataset, sometimes it will predict words that it deems to be fit.


### Importing all the necessary libraries
1. PyAudio - To capture voices from microphone
2. SpeechRecognition - To convert the input captured from microphone into texts
3. Tokenizer- To tokenize the words to be able to predict better
4. pad_Sequences - so that all the inputs are of the same size
5. time - to save the audio file generated with a specific timestamp
6. gTTS - to convert the predicted text into speech
7. Pandas - to import the dataset and convert it into a dataframe

In [1]:
import tensorflow as tf

C:\Users\prith\Anaconda3\envs\tf-gpu\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\prith\Anaconda3\envs\tf-gpu\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\prith\Anaconda3\envs\tf-gpu\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\prith\Anaconda3\envs\tf-gpu\lib\site-pac

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
import speech_recognition as sr
import time

In [4]:
from gtts import *
import os

In [5]:
import pandas as pd
import string

In [6]:
import numpy as np

### Downloading dataset from Kaggle consisting of songs

In [7]:
dataset=pd.read_csv(r'..\Data\songdata.csv')

In [8]:
df = dataset.loc[dataset['artist'].isin(['Coldplay','Sia','Eminem'])]

### Loading a pre-saved model which was built in google colab to predict the words so that this can be given as input to gTTS. gTTS then converts this predicted text to audio file.

In [9]:
reload_sm_keras = tf.keras.models.load_model(r'C:\Users\prith\Desktop\ADS\1591851237')
reload_sm_keras.summary()


Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 19, 32)            132736    
_________________________________________________________________
bidirectional_12 (Bidirectio (None, 19, 128)           49664     
_________________________________________________________________
bidirectional_13 (Bidirectio (None, 128)               98816     
_________________________________________________________________
dense_6 (Dense)              (None, 4148)              535092    
Total params: 816,308
Trainable params: 816,308
Non-trainable params: 0
_________________________________________________________________


In [10]:
def tokenize_corpus(corpus, num_words=-1):
    if num_words > -1:
        tokenizer=Tokenizer(num_words=num_words, oov_token='<OOV>')
    else:
        tokenizer=Tokenizer()
    tokenizer.fit_on_texts(corpus)
    return tokenizer

In [11]:
def create_lyrics_corpus(dataset, field):
  #Removing all punctuations
    dataset[field]=dataset[field].str.replace('[{}]'.format(string.punctuation), '')

  #Make everything lower case
    dataset[field]=dataset[field].str.lower()

  #Make it one long string to split by line
    lyrics=dataset[field].str.cat()
    corpus=lyrics.split('\n')

  #Removing any trailing whitespace
    for l in range(len(corpus)):
        corpus[l]=corpus[l].rstrip()

  #Remove any empty lines
    corpus=[l for l in corpus if l != ' ']

    return corpus

In [12]:
corpus = create_lyrics_corpus(df, 'text')

# Tokenize the corpus
tokenizer = tokenize_corpus(corpus)

total_words = len(tokenizer.word_index) + 1

print(tokenizer.word_index)
print(total_words)
print(corpus)

{'i': 1, 'you': 2, 'the': 3, 'to': 4, 'and': 5, 'a': 6, 'me': 7, 'my': 8, 'it': 9, 'in': 10, 'im': 11, 'of': 12, 'on': 13, 'your': 14, 'that': 15, 'oh': 16, 'is': 17, 'dont': 18, 'so': 19, 'for': 20, 'all': 21, 'but': 22, 'its': 23, 'be': 24, 'this': 25, 'just': 26, 'up': 27, 'like': 28, 'we': 29, 'no': 30, 'with': 31, 'when': 32, 'what': 33, 'know': 34, 'see': 35, 'now': 36, 'get': 37, 'do': 38, 'youre': 39, 'go': 40, 'can': 41, 'got': 42, 'if': 43, 'never': 44, 'are': 45, 'was': 46, 'not': 47, 'yeah': 48, 'down': 49, 'want': 50, 'love': 51, 'come': 52, 'they': 53, 'one': 54, 'out': 55, 'from': 56, 'as': 57, 'have': 58, 'at': 59, 'cause': 60, 'will': 61, 'back': 62, 'ill': 63, 'how': 64, 'he': 65, 'let': 66, 'chorus': 67, 'life': 68, 'cant': 69, 'say': 70, 'time': 71, 'way': 72, 'gonna': 73, 'her': 74, 'shit': 75, 'feel': 76, 'only': 77, 'aint': 78, 'away': 79, 'then': 80, 'she': 81, 'there': 82, 'here': 83, 'take': 84, 'thats': 85, 'think': 86, 'ive': 87, 'baby': 88, 'off': 89, 'were

C:\Users\prith\Anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\prith\Anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [36]:
def red():
    seed_text = r.recognize_google(audio_text)
    next_words = 7  # no. of words to predict after 
  
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]#tokenizing the speech input of user
        token_list = pad_sequences([token_list], maxlen=19, padding='pre')# padding it for same length
        predicted = np.argmax(reload_sm_keras.predict(token_list), axis=-1)# selecting the the token with highest value 
        output_word = ''
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
            
        seed_text += " " + output_word
    print(seed_text)    
    
    text=(seed_text)
    language='en'
    speech = gTTS(text = text, lang = language, slow = False)
    t=time.time()
    saved_as='./{}.mp3'.format(int(t))
    speech.save(saved_as)
    return os.system('start' +saved_as)




In [40]:
r = sr.Recognizer()

with sr.Microphone() as source:
    print('Talk')
    audio_text = r.record(source, duration=4)
    print('Time over, thanks!')

    try:
        print('Text:'+r.recognize_google(audio_text))
    except:
        print('Sorry, didnt get that')
        
st=''.join(r.recognize_google(audio_text))

Talk
Time over, thanks!
Text:sky full of stars


In [43]:
red()

0

## Conclusion

Not bad for a model with 83% accuracy. We could have acheieved more, but given the hardware restrictions and the amount of time consumed to train the network I believe you get the basic idea. It was able to successfully predict the lyrics in the song 'A Sky Full of Stars' by Coldplay. But if you give it a thought, imagine what a model with 95% accuracy can acheive.

And, to all the curious minds out there, this is just a basic level understanding of how Alexa, Siri, Google Home etc. function.

## References
1. https://towardsdatascience.com/easy-text-to-speech-with-python-bfb34250036e
2. The Tensorflow Author
3. https://towardsdatascience.com/convert-your-speech-to-text-using-python-1cf3eccfa922
4. https://colab.research.google.com/drive/1Hgxl6JWOuTnbuIXYrTLabTzvx2CSstLG#scrollTo=IYB2zxaczkQf

## License
Copyright 2020 PRITHVIRAJ PATIL

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:
The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.